In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# attn_idc: [batch_size, num_classes]
# attn_weights: [batch_size, max_seg_length, n_attns]

In [ ]:
eps = 1e-6

In [ ]:
batch_size, num_classes, max_seg_length, max_seg_label_length, n_attns = 2, 6, 7, 6, 3

In [ ]:
seg_labels = torch.LongTensor([[6, 6, 4, 1, 1, 4], [2, 2, 1, 2, 0, 0]])
seg_times = torch.LongTensor([[3, 7, 2, 1, 2, 3], [2, 6, 3, 4, 0, 0]])

In [ ]:
attn_weights = torch.randn(batch_size, max_seg_length, n_attns)
attn_weights

In [ ]:
# attn_weights: [batch_size, seg_length, n_attns]
attn_weights = F.softmax(attn_weights, dim=1)
attn_weights

In [ ]:
# attn_idc: [batch_size, num_classes]
attn_idc = torch.randint(n_attns, (batch_size, num_classes))
attn_idc

In [ ]:
zeros = torch.zeros(batch_size, 1).long()
zeros

In [ ]:
attn_idc = torch.cat((zeros, attn_idc), dim=1)
attn_idc

In [ ]:
#seg_labels = torch.LongTensor([[6, 6, 4, 1, 1, 4], [2, 2, 1, 2, 0, 0]])
#seg_times = torch.LongTensor([[3, 7, 2, 1, 2, 3], [2, 6, 3, 4, 0, 0]])

In [ ]:
# selected_attn_idc: [batch_size, max_seg_label_length]
selected_attn_idc = torch.gather(attn_idc, 1, seg_labels)
selected_attn_idc

In [ ]:
mask = seg_labels.float().ge(0.5)
mask

In [ ]:
# attn_weights: [batch_size, n_attns, seg_length]
attn_weights = attn_weights.transpose(1, 2)
attn_weights

In [ ]:
# Batched index_select
def batched_index_select(t, dim, inds):
    dummy = inds.unsqueeze(2).expand(inds.size(0), inds.size(1), t.size(2))
    out = t.gather(dim, dummy) # b x e x f
    return out

In [ ]:
# selected_attn_weights: [batch_size, max_seg_label_length, max_seg_length]
selected_attn_weights = batched_index_select(attn_weights, 1, selected_attn_idc)
selected_attn_weights

In [ ]:
#seg_labels = torch.LongTensor([[6, 6, 4, 1, 1, 4], [2, 2, 1, 2, 0, 0]])
#seg_times = torch.LongTensor([[3, 7, 2, 1, 2, 3], [2, 6, 3, 4, 0, 0]])

In [ ]:
zeros = torch.zeros(batch_size, max_seg_label_length, 1)

In [ ]:
selected_attn_weights = torch.cat((zeros, selected_attn_weights), dim=2)
selected_attn_weights

In [ ]:
# seg_times: [batch_size, max_seg_label_length, 1]
seg_times = seg_times.unsqueeze(2)
seg_times

In [ ]:
all_seg_times = torch.arange(max_seg_length+1)
all_seg_times

In [ ]:
all_seg_times = all_seg_times.expand(batch_size, max_seg_label_length, max_seg_length+1)
all_seg_times

In [ ]:
probs = torch.where(all_seg_times == seg_times, selected_attn_weights, 1-selected_attn_weights)
probs

In [ ]:
probs = probs[:, :, 1:]
probs

In [ ]:
loss = -1 * torch.log(probs + eps).sum()
loss

In [ ]:
loss_size = batch_size * max_seg_label_length * max_seg_length
loss_size